# Overview over project files
* Documentation/writeup in FindingLaneLines.pdf
* My local results on videos and images in folder: MyResultsVideosAndImages
* Source code written with Eclipse-Pydev in folder (using cv2 instead of moviepy for video processing): MySource
* Writeup pictures and WORD document in folder: MyWriteup


## Import Packages

In [1]:
#importing some useful packages
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import matplotlib.pyplot as plt
import math
import glob, os
%matplotlib inline

## Pipeline
My (final) pipeline consists of several steps which are 
1. Converting the image from BGR space to HSV space
2. Applying a yellow and a white color mask 
3. Converting the HSV image to a gray colored image
4. Applying a smoothing of the picture to reduce noise effects.
5. Apply an edge detection algorithm, here the Canny algorithm.
6. Defining a region of interest, where lines forming road lanes should be identified.
7. Using the Hough transform to identify lines in the image
8. Using a linear/quadratic fit to identify the lane lines.
9. Draw the resulting lines in the picture
    * Draw one line for each lane line only implemented for linear fit.
    * Draw line segments for quadratic fit 
    
### Problems with color spaces in moviepy and cv2
I encounterd a problem when I used movie.py instead of cv2 (my original choice) to capture the videos. To see the difference see the different colors (red and blue) in test-videos and test-images. In both cases I work with RGB but in cv2 I get blue colors while I get red colors using moviepy. It seems that RGB and BGR a defined differently in the two libraries. As result my challenge-video and the solidYellowLeft video didn't work at beginning in jupyter-notebook and only showed one blue lane line instead of two red lane lines.
IN my IDE (eclipse-pydev) I used cv2 and BGR2GRAY. To get the same results using moviepy and jupyter I had to adjust all my color transformations and use e.g. RGB2GRAY instead of BGR2GRAY. 



In [2]:
'''
Created on Sep 11, 2017

@author: andre
'''

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import glob, os



'''
This class is used for the purpose to identify yellow and white structures on an image 
and perform the following transformations
RGB -> GRAY
RGB -> HSV
HSV -> GRAY 
'''
class ColorTransformations(object):
    def ConvertBGRImageToGrayColorSpace(self,image):
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    def ConvertImageToHSVSpace(self, image):
        return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    def ConvertHSVImageToGrayColorSpace(self, image):
        image1 = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        return self.ConvertBGRImageToGrayColorSpace(image1)

    def YellowHSVMask(self,image):
        yellowHSVLow  = np.array([ 0, 80, 200])
        yellowHSVHigh = np.array([ 40, 255, 255])
        return cv2.inRange(image,yellowHSVLow, yellowHSVHigh)
        
    def WhiteHSVMask(self,image):
        whiteHSVLow  = np.array([  20,   0,   200])
        whiteHSVHigh = np.array([ 255,  80, 255])
        return cv2.inRange(image,whiteHSVLow, whiteHSVHigh)
    
    def ApplyWhiteAnYellowColorMasks(self, image):
        yellowMask = self.YellowHSVMask(image)
        whiteMask = self.WhiteHSVMask(image)
        mask = cv2.bitwise_or(yellowMask, whiteMask)
        filtered = cv2.bitwise_and(image, image, mask = mask)
        return filtered

'''
This is the central class which processes all images.
'''
class LaneRecognition(object):
    '''
     Constructor 
     '''
    def __init__(self):
        self.Color = ColorTransformations

    def ApplyCannyEdgeDetection(self, image, lowerThreshold=50, upperThreshold=150):
        return cv2.Canny(image, lowerThreshold, upperThreshold)
        
    def ApplyGaussianSmoothing(self, image, kernelSize =5):
        return cv2.GaussianBlur(image,(kernelSize, kernelSize),0)
    
    
    def YTetragonBorder(self, yImageSize, yRegionRatio):
        return yImageSize - (int (float (yImageSize) * yRegionRatio))
    
    '''
    Generates a tetragon-mask based on the image size and the following two inputs 
    yRegionRatio: Gives the ratio of the considered image in y direction, which will be used to build the tetragon  
    xQuadTopWidth: Gives the full width of the tetragon at the top. BottomWidth is full x-range of image  
    The mask is applied bitwise to the input-image and the result is returned
    '''
    def DefineTetragonROIAndApplyToImage(self, image, yRegionRatio, xTetragonTopWidth = 80):
        yImage = image.shape[0]
        xImage = image.shape[1]
        mask = np.zeros_like(image)
        xTopLeft = xImage/2 - xTetragonTopWidth/2
        xTopRight = xImage/2 + xTetragonTopWidth/2
        xBottomLeft = 0
        xBottomRight = xImage
        yBottom = yImage
        yTetragonBorder = self.YTetragonBorder(yImage, yRegionRatio)
        tetragonEdges = np.array([[(xBottomLeft, yBottom), (xTopLeft, yTetragonBorder), (xTopRight, yTetragonBorder), (xBottomRight, yBottom)]], dtype= np.int32)
        cv2.fillPoly(mask, tetragonEdges, 255)
        result = cv2.bitwise_and(image,mask)
        return result
    
    
    '''
     Angular resolution converted in radians, input in degrees
    '''
    def ProbalisticHoughTransform(self, image, distanceResolution =2, angularResolution=1, numberOfPointsDefiningALine = 15, minimalLineLength = 20, maximalLineGap=20):
        angularResolution = angularResolution*np.pi/180.0
        identifiedLines = cv2.HoughLinesP(image, distanceResolution, angularResolution, numberOfPointsDefiningALine, np.array([]), minimalLineLength, maximalLineGap)
        return identifiedLines
    
    '''
    Draw lines into an empty image 
    '''
    def CreateLineImage(self, lines, emptyImage):
        for line in lines:
            for x1,y1,x2,y2 in line:
                '''
                Check if length of lines is a good discriminator 
                Answer: No
                x = x1 - x2
                y = y1 - y2
                res = (x**2 + y**2)**.5
                if(res > 120):
                    cv2.line(emptyImage,(x1,y1),(x2,y2),(0,0,255),10)
                elif(res > 80):
                    cv2.line(emptyImage,(x1,y1),(x2,y2),(0,255,0),10)
                else:
                    cv2.line(emptyImage,(x1,y1),(x2,y2),(0,0,255),10)
                '''
                cv2.line(emptyImage,(x1,y1),(x2,y2),(255,0,0),10)
        return emptyImage
                
    
    '''
    Reduce the number of lines which are not in the direction of the lanes.
    Central angle is choosen to be 45 degree. Default gap is 15 degree 
    '''
    def ApplyAngleFiltering(self, lines, centralAngle = 45, angleGap=15):
        lowerAngle = math.radians(centralAngle-angleGap)
        upperAngle = math.radians(centralAngle+angleGap) 
        filteredLines = []
        if lines is None:
            return filteredLines
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2-y1)/(x2-x1)
                angle = abs(math.atan(slope))
                if(angle > lowerAngle and angle < upperAngle):
                    filteredLines.append( [[x1, y1, x2, y2]] )
        return filteredLines

    
    '''
    We have two lanes and consequently two categories for all lines: 
    a) Lines with negative slope represent the left lane (coordinate system of image)
    b) Lines with positive slope represent the right lane
    Does a linear assumption using the mean values of left/right lines work?
    Answer: NO, not good enough. Resulting lanes are flickering/unstable and detection in video 'challenge.mp4' is bad
    '''
    def IdentifyLanes(self, lines, yImageLowerBorder, xImageRightBorder, yRegionRatio):
        leftLane = [[0,0,0,0]]
        rightLane = [[0,0,0,0]]
        leftLaneSlope = 0
        rightLaneSlope = 0
        
        leftLaneLineSlopes = [] 
        rightLaneLineSlopes = []
        rightLaneLineIntercepts = []
        leftLaneLineIntercepts = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if(abs(x2-x1) < 1.e-3):
                    continue
                slope = float(y2-y1)/float(x2-x1)
                xCenter = (int )(xImageRightBorder/2)
                if( slope < 0):
                    leftLaneLineSlopes.append(slope)
                    leftLaneLineIntercepts.append(y1-slope*x1)
                    
                elif(slope >0):
                    rightLaneLineSlopes.append(slope)
                    rightLaneLineIntercepts.append(y1-slope*x1)
                    
        # Calculate the mean values of slope and intercept for both lanes  
        if(len(leftLaneLineSlopes) > 0):
            leftLaneSlope = np.mean(leftLaneLineSlopes)
            leftLaneIntercept = np.mean(leftLaneLineIntercepts)
        if(len(rightLaneLineSlopes) > 0):
            rightLaneSlope = np.mean(rightLaneLineSlopes)
            rightLaneIntercept = np.mean(rightLaneLineIntercepts)
         
        yTetragonBorder = self.YTetragonBorder(yImageLowerBorder, yRegionRatio)
        
        y1Left = yImageLowerBorder
        y1Right = y1Left
        y2Left = (int) (yTetragonBorder)
        y2Right = y2Left
        
        if(abs(leftLaneSlope) > 1.e-8 ):
            x1Left = (int) ((yImageLowerBorder-leftLaneIntercept)/leftLaneSlope)
            x2Left = (int) ((yTetragonBorder-leftLaneIntercept)/leftLaneSlope)
            leftLane = [[x1Left, y1Left, x2Left, y2Left]]
            
        if(abs(rightLaneSlope) > 1.e-8 ):
            x1Right = (int) ((yImageLowerBorder-rightLaneIntercept)/rightLaneSlope)
            x2Right = (int) ((yTetragonBorder-rightLaneIntercept)/rightLaneSlope)
            rightLane = [[x1Right, y1Right, x2Right, y2Right]]
            
        
        return [leftLane, rightLane]
    
    '''
    Create fit function from (x,y) value pairs and return new y values
    '''
    def CalculateLanePoints(self, x,y,degree):
        try:
            laneFit = np.polyfit(x,y,degree)
            laneFunction = np.poly1d(laneFit)
            return laneFunction(x).astype(int)
        except TypeError:
            return np.ndarray([0])
    
    '''
    Using polynomial fit instead of simple linear fit
    - Use points of all lines and assign them according to the x-Value to left and right lane
    - Make fit and define function
    Result is not yet satisfying, since the right lane segments are not connected.
    '''
    def IdentifyLinesWithPolynom(self, lines, xImageRightBorder, polynomDegree =2):
        leftLaneLineX = []
        leftLaneLineY = []
        rightLaneLineX = []
        rightLaneLineY = []
        xCenter = (int )(xImageRightBorder/2)
        leftLanePoints = np.column_stack(( [], [] )).reshape(-1,1,2)
        rightLanePoints = np.column_stack(( [], [] )).reshape(-1,1,2)
        if(lines is None):
            return leftLanePoints, rightLanePoints

        for line in lines:
            for x1, y1, x2, y2 in line:
                if( x1 < xCenter and x2 < xCenter):
                    leftLaneLineX.append(x1)
                    leftLaneLineY.append(y1)
                    leftLaneLineX.append(x2)
                    leftLaneLineY.append(y2)

                elif( x1 > xCenter and x2 > xCenter):
                    rightLaneLineX.append(x1)
                    rightLaneLineY.append(y1)
                    rightLaneLineX.append(x2)
                    rightLaneLineY.append(y2)

        
        newLeftLaneY = self.CalculateLanePoints(leftLaneLineX, leftLaneLineY, polynomDegree)
        newRightLaneY= self.CalculateLanePoints(rightLaneLineX, rightLaneLineY, polynomDegree)
        if not newLeftLaneY.any():
            leftLaneLineX = []
        if not newRightLaneY.any():
            rightLaneLineX = []
        leftLanePoints = np.column_stack(( np.asarray(leftLaneLineX), newLeftLaneY )).reshape(-1,1,2)
        rightLanePoints = np.column_stack((np.asarray(rightLaneLineX), newRightLaneY)).reshape(-1,1,2)
        
        return leftLanePoints, rightLanePoints 
        
    '''
    Create not connected Polygon with cv2 
    '''
    def CreatePolygonLines(self, image, leftPoints, rightPoints):
        image1 = cv2.polylines(image,[leftPoints],False,(255,0,0),10)
        return cv2.polylines(image1,[rightPoints],False,(255,0,0),10)
        
    



    def ProcessImage(self, image, linear):
        yRegionRatio = 0.4
        '''
        ColorTransformations to identify white and yellow objects
        '''
        color = self.Color
        hsv = color().ConvertImageToHSVSpace(image)
        whiteAndYellow = color().ApplyWhiteAnYellowColorMasks(hsv)
        gray = color().ConvertHSVImageToGrayColorSpace(whiteAndYellow)
        #gray = color().ConvertBGRImageToGrayColorSpace(image)
        
        '''
        Smoothing, CannyEdge, ROI
        '''  
        smoothie = self.ApplyGaussianSmoothing(gray)
        edges = self.ApplyCannyEdgeDetection(smoothie)
        roi = self.DefineTetragonROIAndApplyToImage(edges, yRegionRatio = yRegionRatio)
        
        '''
        Line identification via HoughTransform and extrapolation.
        Two methods are possible (depending on the video): 
        - Simple linear fit with angle filtering
        - Polynomial fit for challenge-video
        Goal must be to use one global function or a decision maker to decide when to use which function
        '''        
        identifiedLines = self.ProbalisticHoughTransform(roi)
        emptyImage = np.copy(image)*0 # Empty frame for lines
        #lineImage = self.CreateLineImage(identifiedLines, emptyImage)
        
        if(linear):
            identifiedLines = self.ApplyAngleFiltering(identifiedLines)
            twoLanes = self.IdentifyLanes(identifiedLines, emptyImage.shape[0],
                                          emptyImage.shape[1], yRegionRatio = yRegionRatio)
            lineImage = self.CreateLineImage(twoLanes, emptyImage)
        else:
            leftPoints, rightPoints = self.IdentifyLinesWithPolynom(identifiedLines, emptyImage.shape[1], polynomDegree=2)
            lineImage = self.CreatePolygonLines(emptyImage, leftPoints, rightPoints)
        
        result = cv2.addWeighted(image, 0.8, lineImage, 1, 0)
        
        return result
    
    

    
    
    
class ImageLaneFinder(object):
    def __init__(self, path):
        self.Processor = LaneRecognition()
        self.Path = path
    
    def LaneImages(self):
        for file in os.listdir(self.Path):
            if file.endswith(".jpg") and not file.endswith("_result.jpg"):
                image = cv2.imread(self.Path+file)
                result = self.Processor.ProcessImage(image, True)
                outputPath = (self.Path+file).replace(".jpg", "_result.jpg")
                cv2.imwrite(outputPath, result)
            
            
 
    




## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:

path = str(os.getcwd())+"\\test_images\\"

Images = ImageLaneFinder(path)
Images.LaneImages()


D:\Andreas\Programming\Python\UdacitySelfDrivingCar\Term1Projects\Project1\CarND-LaneLines-P1\test_images\


## Test videos
If the test videos show blue instead of red lines, please read my comments about the color space problem at the beginning of the document.

### First video: solidWhiteRight
My original output video can be found in the folder MyResultsVideosAndImages: solidWhiteRightMyOutput.mp4

In [5]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
print(str(os.getcwd()))

LR =LaneRecognition()
clip1 = VideoFileClip('test_videos/solidWhiteRight.mp4')
#apply LaneFinder on videoclip
white_clip = clip1.fl_image(lambda x: LR.ProcessImage(x, True))
%time white_clip.write_videofile(white_output, audio=False)

D:\Andreas\Programming\Python\UdacitySelfDrivingCar\Term1Projects\Project1\CarND-LaneLines-P1
[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:04<00:00, 53.98it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 4.48 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

### Second video: solidYellowLeft
My original output video can be found in the folder MyResultsVideosAndImages: solidYellowLeftMyOutput.mp4

In [7]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
print(str(os.getcwd()))


clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
#apply LaneFinder on videoclip
yellow_clip = clip2.fl_image(lambda x: LR.ProcessImage(x, True))
%time yellow_clip.write_videofile(yellow_output, audio=False)

D:\Andreas\Programming\Python\UdacitySelfDrivingCar\Term1Projects\Project1\CarND-LaneLines-P1
[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:12<00:00, 52.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 13.4 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

### Third video: challenge
My original output video can be found in the folder MyResultsVideosAndImages: challengeMyOutput.mp4
Please read comments in the writeup: FindingLaneLines.pdf

In [9]:
c_output = 'test_videos_output/challenge.mp4'
print(str(os.getcwd()))


clip3 = VideoFileClip('test_videos/challenge.mp4')
#apply LaneFinder on videoclip
c_clip = clip3.fl_image(lambda x: LR.ProcessImage(x, False))
%time c_clip.write_videofile(c_output, audio=False)

D:\Andreas\Programming\Python\UdacitySelfDrivingCar\Term1Projects\Project1\CarND-LaneLines-P1
[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:09<00:00, 24.27it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 10.3 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(c_output))